# Text Generation
Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [5]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 34456
total chars: 50


In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

nb sequences: 11472
Vectorization...
Build model...
Epoch 1/60
11472/11472 [==============================] - 9s 806us/step - loss: 2.9843

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ve for a hardy, laborious race of machin"
ve for a hardy, laborious race of machin ans an an in senin in ane therin on in an thes an an the thesins an ins an in thes an ine the in thereres ans an the therererese thes an thes an ans she thes an in an thes ing on thererer in the the the thentherens ins ine therer thes in thes in in in thes in thesis thes and thes an thens an thes an the the sis an there thes an the thes an therese theses an the in these therthere therenses thes a
----- diversity: 0.5
----- Generating with seed: "ve for a hardy, laborious race of machin"
ve for a hardy, laborious race of maching in is in the phanthenilhels phe thesanely her in rous on antals an  thenin thelins an al aan endiwey thet an lingns thenlantilesssan ind thesannel phenthere

abbreviation of the mendides propeduf.dernay.-inters ios preally a pecousois byopelior fopres tout "themsitleapr icurt selto ffilos fore ard steofke constiol morerenay and bothar arhsucare yogerel of er tce mates ropaly, ow balced are re!orey the f ore, what hahes uce furstowingn ildercale--a dourkely abe whad "the itcesuin, an antur menebtef of thote foriale momestalayyen bo dad arefuiserer lout as mirefure
fast," wo 
----- diversity: 1.2
----- Generating with seed: "eryday use (as an
abbreviation of the me"
eryday use (as an
abbreviation of the messers avey ap mane! foul"
yof ner somentn ofs! ut momeay ead condars, of hanrig tien, pheverong ohichey line tyrese-. dads, ouse a craencobed bly rose, tithe, wemat
ychawcreat yunne
engale
abendditbiony yfurdint allus char daan suanaal, it sendet nos "fsenongas suss sbewity? the "tand.us oopany they, it ther
ieceeven, isnkeraty mosifedrle core, lres acgimy not, duertimvecy orcar of
wha custely of

Epoch 5/60
11472/11472 [====================

he who hears only a "will of the mast the persalives of the most the more of the persaled the prestion of the mast the more of the prean stolled to bele preans of as as as as as as as as ofer as an all the mast of the forld the most of the more of the mast of the senses of the most the plolostion of the most of the presess of the preang to the still of the palles the senses of the prestion of the prestion of the sthent of 
----- diversity: 0.5
----- Generating with seed: "ttention; and
he who hears only a "will "
ttention; and
he who hears only a "will the senses of is more of the perses of the forses. it mast at ustance prilosophy in the renestion of ther of the sement of the persely appered dowhers of the prisestion of the menaling to has as oples evervent the whoch to the most of the world the sensed to deemfoons and is not the prose to persalives as as the "conses of the has and most
that the preas of thes the ait is as is not the fore of th
----- diversity: 1.0
----- Generating wi

persled, wis -beingly, to gerant to, degay on a than in imcopnicion; it sucs of wore have-bue to been in inword endervert solled by the fact,

a priably the
----- diversity: 1.2
----- Generating with seed: "ngth, swing, courage, and artistic power"
ngth, swing, courage, and artistic powerst
of (an inkart presence,
podre nate
in the worls, midedo pirsund insticcrese-to be hive atselved wore and hyporsanges of igseas
so its
conby,
pliits in the pensare, to e
corly
wist? plow ndearst, to winl adsion to gouls bee ayverabnet, invermuncatlyciounate, in
toren?--to desedvay
pnemicions--in therpins! and you was irngingare furm cugcheb, yow for
no ward us" be mide, to atevers ofive usvelven
Epoch 12/60
11472/11472 [==============================] - 8s 674us/step - loss: 1.5249

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "at thinking is an activity and operation"
at thinking is an activity and operation of the the ware the wark, there in the cansing the

unain on the fartley accuad on the farther of think, it se manter, the ast man live hevertion of the sense that there it, the wand to the whoch thith at is nithing porely, te preans the sentary hacule, that is have in this "more, sthit them the wande of which the warking that is bele the whine thought, and the prople the with canionstand have haves
pussilly and "putsention, and the belien and some t
----- diversity: 1.0
----- Generating with seed: " it otherwise--and to crown all, some
un"
 it otherwise--and to crown all, some
und enduidion of the consiens bethew, reteartic prounce, al it wourd nature of think, whill es thinkiun the moretrua cenered roggrusion, scongale
theus fuery, thith, the will wecale ther centainces--olly "who dace tame-philo of the remant conucaly and abvink! accured, and out charsphating"! stlelf
yout nothinge? a ware of a wayded whonuthoubyour ans acces and ad, ate thers; in extruthentical in, whi
----- diversity: 1.2
----- Generating with seed: " it otherwise-

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log


emustable?". of
th) all everanoshes bedincie
tiut than lifferegily of
thes"! folfistio
Epoch 19/60
11472/11472 [==============================] - 9s 774us/step - loss: 1.1088

----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: "nder. "where there is nothing more to se"
nder. "where there is nothing more to seem notwisust endessubyes of the senses in such to speal yould be plaining of the supersing of which mose for the sopers of the supersision of spectary ast and about of way more for leven that is hindsess of the sthinition of the scectary spocking to be most the superstitions of the senses of cus penses of cus to speal yous and mosesments of the sentapess to speice of the sentrestien the senses of 
----- diversity: 0.5
----- Generating with seed: "nder. "where there is nothing more to se"
nder. "where there is nothing more to seem to speally spichtress of "lifted and of the sible "persunte the spopuntion has presentansions,
a deants bous and accua

KeyboardInterrupt: 